In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import StructType,StructField, StringType
from awsglue.context import GlueContext
from awsglue.job import Job
import boto3
from datetime import datetime, timedelta
import pytz
from dateutil import tz

utc=pytz.UTC

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

s3 = boto3.client("s3")

##Config

#input definitions

addin_bucket = "credence-cloudwatch-metrics-addins"
data_bucket ="credence-cloudwatch-metrics-raw"
report_bucket = "credence-cloudwatch-metrics-reports"
main_prefix = "ec2-metrics/"

# Sets up automatic date generation for file sorting and destination naming
now = datetime.now()
first_day_of_current_month = datetime(now.year, now.month, 1)

last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)
month = last_day_of_previous_month.strftime('%b')

year_num = now.year
month_num = now.month-1 # two digit month'08'

In [ ]:
# Accounts for January getting last month of previous year

if now.month == 1:
    month_num = 12
    year_num = year_num - 1

# Accounts for the required two digits on one digit months
elif now.month < 11:
    month_num = f"0{str(month_num)}"

# Ensures string type for month and year
year_num = str(year_num)
month_num = str(month_num)

 

# month_num = '11'

# month = 'Nov'

print(month, month_num)
rnd = 0 # Number of digits to round to

start_date = f"{year_num}-{month_num}-01"
end_date = f"{year_num}-{month_num}-31" # No need to change

# Patching has to be entered manualy every month
start_ts = "2025-03-04 00:08:00" # Patching start/end time 
end_ts = "2025-03-04 03:13:00"

##Config 8:30EST == 13:30GMT

patch_start_dt = datetime.strptime(start_ts, "%Y-%m-%d %H:%M:%S")
patch_end_dt = datetime.strptime(end_ts, "%Y-%m-%d %H:%M:%S")
total_patch_time = (patch_end_dt - patch_start_dt).total_seconds()/3600

In [ ]:
dest_report = f"s3://{report_bucket}/Uptime_Reports/Processed/{year_num}/{month_num}" #daas_daasebusgexd93_
dest_report_raw = f"s3://{report_bucket}/Raw/{year_num}/{month_num}"

tenant_list = ["daas", "daas-api","daas-aiml"]
env_list = ["prod"] # api-prod

print(f"tenant list: {tenant_list}")

In [ ]:
main_dict = {}
##Config

col_list = [
    'instance_id',
    'instance_name',
    'instance_type',
    'environment',
    'DATE',
    'TIME',
]

 
for tenant in tenant_list:
    print(f"Environment list: {env_list}")
    tenant_dict = {}

    # #Iterate through each environment within the tenant, get csv list, prep to iterate and insert dataframe into dict
    for env in env_list:
        print(data_bucket + '/' + main_prefix + tenant + '/' + env)

        tenant_dict[env] = {'Raw': (spark.read.options(header='True', inferSchema='True', delimiter=',')
            .csv(f"s3://{data_bucket}/{main_prefix}{tenant}/{env}/")
            .filter(col("instance_type") != "terminated_instance")
            .filter(col("DATE").between(start_date,end_date))
            .withColumn("input_file_name",input_file_name())
            .withColumn("dt_string", concat('DATE', lit(' '), "TIME"))
            .withColumn("timestamp", col("dt_string").cast("timestamp"))
            .withColumn("environment", regexp_replace("environment","-","_"))
            .withColumn("environment", regexp_replace("environment","piee_",""))
            .filter((

                (col("environment") != "daas_prod")

                | (

                    (col("environment") == "daas_prod")

                    & (~col("timestamp").between(patch_start_dt, patch_end_dt))

                    )

                )

            )

            .withColumn("instance_name", regexp_replace("instance_name","-d","d"))
            .withColumn("host_name", regexp_replace("host_name","\['",""))
            .withColumn("host_name", regexp_replace("host_name","'\]",""))

            #.filter(col("timestamp").isNotNull())

            # .filter(col("instance_name") == "daasebusgexd93 - RHEL8") #testing 3/3/23

        )}

        tenant_dict[env]['Raw'].printSchema()

    main_dict[tenant] = tenant_dict


groupby_col_list = [
    col("instance_id"),
    col("instance_name"),
    col("environment"),
    col("instance_type"),
    col("host_name")

]

agg_list = [
    count("*").alias("Recorded Uptime"),
    min(col("timestamp")).alias("First Recorded Timestamp"),
    max(col("timestamp")).alias("Last Recorded Timestamp"),
    when(
        col("Environment") == "daas_prod",
        patch_start_dt
    ).alias("Planned Patching Start"),
    when(
        col("Environment") == "daas_prod",
        patch_end_dt
    ).alias("Planned Patching End"),
    round(((max(col("timestamp")).cast("long") - min(col("timestamp")).cast("long") + 300) / 3600),rnd).alias("Total Availability (Hours)"),
    when(
        ((col("Environment") == "daas_prod") & ((min(col("timestamp")) < patch_start_dt) & (max(col("timestamp")) > patch_end_dt)))

        ,total_patch_time

    ).alias("Patching Downtime (Hours)"),

    (
        (
            (max(col("timestamp")).cast("long") - min(col("timestamp")).cast("long") + 300) / 3600

        ) - when(

                (col("Environment") == "daas_prod") &

                (
                    (min(col("timestamp")) < patch_start_dt) &

                    (max(col("timestamp")) > patch_end_dt)

                ),

                total_patch_time

        ).otherwise(0)

    ).alias("Total Availability Excluding Patching (Hours)")

]

In [ ]:
for tenant in main_dict:
    for env in main_dict[tenant]:
        print(tenant + " " + env)
        temp_raw_df = (main_dict[tenant][env]['Raw']) #.join(daas_zone_df, "instance_id","left"))
        temp_metrics_df = (main_dict[tenant][env]['Raw']
        .orderBy(*groupby_col_list,col("timestamp"))
        .groupBy(groupby_col_list + [lit(tenant).alias("Program")])
        .agg(*agg_list)

        )

        if('main_df' not in locals()):
            raw_df = temp_raw_df
            main_df = temp_metrics_df
        else:
            main_df = main_df.unionByName(temp_metrics_df,allowMissingColumns=True)
            raw_df = main_df.unionByName(temp_raw_df,allowMissingColumns=True)


main_df = (main_df.withColumn('Recorded Uptime',round(col('Recorded Uptime')/12, rnd))
            .withColumnRenamed("instance_id", "Instance ID")
            .withColumnRenamed("instance_name", "Instance Name")
            .withColumn("Instance Name",
                when(col("Instance Name").startswith("="), expr("substring('Instance Name', 2, length('Instance Name') - 1)"))
                .when(col("Instance Name").endswith("-"), expr("substring('Instance Name', 1, length('Instance Name') - 1)"))
                .otherwise(col("Instance Name")))
            .withColumnRenamed("environment", "Environment")
            .withColumn(
                "IP Address",
                when(col("host_name") == '[None]', None)
                .when(
                    col("host_name").rlike(r"ip-\d{1,3}-\d{1,3}-\d{1,3}-\d{1,3}"),
                    regexp_replace(
                        regexp_extract(col("host_name"), r"ip-(\d{1,3}-\d{1,3}-\d{1,3}-\d{1,3})", 1),
                        "-", "."
                    )

                )
                .otherwise(None)

            )

            .withColumnRenamed("instance_type", "Instance Type")
            .withColumn(
                "Patching Downtime (Hours)",
                when(col("Planned Patching End").isNotNull(),
                    round((col("Planned Patching End").cast("long") - col("Planned Patching Start").cast("long")) / 3600,rnd)
                ).otherwise(None)

            )

            .withColumn("First Recorded Timestamp", date_format(col("First Recorded Timestamp"), "yyyy-MM-dd HH:mm:ss"))
            .withColumn("Last Recorded Timestamp", date_format(col("Last Recorded Timestamp"), "yyyy-MM-dd HH:mm:ss"))
            .withColumn("Planned Patching Start", date_format(col("Planned Patching Start"), "yyyy-MM-dd HH:mm:ss"))
            .withColumn("Planned Patching End", date_format(col("Planned Patching End"), "yyyy-MM-dd HH:mm:ss"))
            .withColumn(
                "Total Availability Excluding Patching (Hours)",
                when(col("Instance Name").endswith('api'),lit('N/A'))
                .otherwise(round(col("Total Availability Excluding Patching (Hours)"),rnd))

            )
            .withColumn(
                "Unplanned Downtime",
                when(col("Instance Name").endswith('api'),lit('N/A'))
                .otherwise((round(col("Total Availability Excluding Patching (Hours)"),2) - round(col("Recorded Uptime"), 2)))

            )
            .withColumn(
                "Percent Uptime",
                when(col("Total Availability Excluding Patching (Hours)") == 'N/A', lit('N/A'))
                .otherwise(round(col("Recorded Uptime") / col("Total Availability Excluding Patching (Hours)"), 2))

            )
        )

main_df.coalesce(1).write.option("header","true").mode("overwrite").csv(dest_report) #.filter(((col("environment") == 'daas_prod') | (col("environment") == 'daas_api_prod')))

 